In [1]:
from typing import List, Dict
import json
import requests
import time

def build_jaeger_trace_endpoint(params : List[str]) -> str:
    return f"http://localhost:6686/api/traces?{"&".join(params)}"

def build_jaeger_spans_endpoint_url(limit: int, service : str, namespace : str, operation : str, lookback_minutes : int, tags : Dict[str,str]) -> str:
    lookback_seconds = 60*lookback_minutes
    start_ms = int((time.time() - lookback_seconds)*1_000_000)
    end_ms = int(time.time()*1_000_000)
    
    tags["service.namespace"] = namespace
    full_tags = "%2C".join([f'"{key}"%3A"{value}"' for (key,value) in tags.items()])
    limit_str = f"limit={limit}"
    lookback_str = f"lookback={lookback_seconds}s"
    service_str = f"service={service}"
    operation_str = f"operation={operation}"
    start_str = f"start={start_ms}"
    end_str = f"end={end_ms}"
    tags_str = f"tags=%7B{full_tags}%7D"
    return build_jaeger_trace_endpoint([limit_str,lookback_str,service_str,operation_str,start_str,end_str,tags_str])

def scrape_jaeger_endpoint(url : str):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        raise err

    print(f"   Response received. Processing json")
    return json.loads(response.text)


In [9]:
#url = build_jaeger_spans_endpoint_url(10,"digitiser-aggregator", "pipeline_1", "Frame", 60,dict());
url = build_jaeger_spans_endpoint_url(10,"digitiser-aggregator", "pipeline_1", "Frame", 60, {"frame_is_expired": "true"});
print(url)
value = scrape_jaeger_endpoint(url)
print(value)

http://localhost:6686/api/traces?limit=10&lookback=3600s&service=digitiser-aggregator&operation=Frame&start=1733097437034272&end=1733101037034273&tags=%7B"frame_is_expired"%3A"true"%2C"service.namespace"%3A"pipeline_1"%7D


   Response received. Processing json
{'data': [{'traceID': '784dd507a7d4413e2759148a5efa0dbc', 'spans': [{'traceID': '784dd507a7d4413e2759148a5efa0dbc', 'spanID': '918d688d70c5975e', 'operationName': 'Frame', 'references': [], 'startTime': 1733101034797877, 'duration': 2005919, 'tags': [{'key': 'otel.library.name', 'type': 'string', 'value': 'opentelemetry-otlp'}, {'key': 'otel.library.version', 'type': 'string', 'value': '0.15.0'}, {'key': 'code.filepath', 'type': 'string', 'value': 'digitiser-aggregator/src/frame/partial.rs'}, {'key': 'code.namespace', 'type': 'string', 'value': 'digitiser_aggregator::frame::partial'}, {'key': 'code.lineno', 'type': 'int64', 'value': 79}, {'key': 'thread.id', 'type': 'int64', 'value': 1}, {'key': 'thread.name', 'type': 'string', 'value': 'main'}, {'key': 'metadata_timestamp', 'type': 'string', 'value': '2024-12-02T00:57:14.753797640+00:00'}, {'key': 'metadata_frame_number', 'type': 'string', 'value': '0'}, {'key': 'metadata_period_number', 'type': '

In [10]:
traces = value['data']
print(json.dumps(traces, indent = 4))

[
    {
        "traceID": "784dd507a7d4413e2759148a5efa0dbc",
        "spans": [
            {
                "traceID": "784dd507a7d4413e2759148a5efa0dbc",
                "spanID": "918d688d70c5975e",
                "operationName": "Frame",
                "references": [],
                "startTime": 1733101034797877,
                "duration": 2005919,
                "tags": [
                    {
                        "key": "otel.library.name",
                        "type": "string",
                        "value": "opentelemetry-otlp"
                    },
                    {
                        "key": "otel.library.version",
                        "type": "string",
                        "value": "0.15.0"
                    },
                    {
                        "key": "code.filepath",
                        "type": "string",
                        "value": "digitiser-aggregator/src/frame/partial.rs"
                    },
                    

In [22]:

url = build_jaeger_spans_endpoint_url(10,"digitiser-aggregator", "pipeline_1", "Frame", 60, {"frame_is_expired": "true"});
result = scrape_jaeger_endpoint(url)
traces = result['data']

for trace in traces:
    num_digitisers = len([span for span in trace["spans"] if span['operationName'] == "Digitiser Event List"])
    frame = next(span for span in trace["spans"] if span['operationName'] == "Digitiser Event List")
    timestamp = next(tag["value"] for tag in frame["tags"] if tag['key'] == "metadata_timestamp")
    timestamp = timestamp.replace("+","%2B").replace(":", "%3A")

    url = build_jaeger_spans_endpoint_url(10,"trace-to-events", "pipeline_1", "process_digitiser_trace_message", 60, {"metadata_timestamp": timestamp});
    result2 = scrape_jaeger_endpoint(url)
    traces2 = result2["data"]
    print(f"num digitisers in aggregator frame: {num_digitisers}, num digitisers in event formation: {len(traces2)}")

   Response received. Processing json
   Response received. Processing json
num digitisers in aggregator frame: 2, num digitisers in event formation: 2
   Response received. Processing json
num digitisers in aggregator frame: 6, num digitisers in event formation: 6
   Response received. Processing json
num digitisers in aggregator frame: 2, num digitisers in event formation: 2
   Response received. Processing json
num digitisers in aggregator frame: 2, num digitisers in event formation: 2
   Response received. Processing json
num digitisers in aggregator frame: 6, num digitisers in event formation: 6
   Response received. Processing json
num digitisers in aggregator frame: 7, num digitisers in event formation: 7
   Response received. Processing json
num digitisers in aggregator frame: 3, num digitisers in event formation: 3
   Response received. Processing json
num digitisers in aggregator frame: 3, num digitisers in event formation: 3
   Response received. Processing json
num digitise